## Prufer

This file translates between adjacency matrices and Prufer sequences, and generates random trees using Prufer sequences.

In [5]:
import numpy as np

### get_prufer(T)

#### Description

Given the adjacency matrix of a tree, return the Prufer sequence as a list.

#### Variables

*T* is the adjacency matrix of a tree.

In [1]:
def get_prufer(T):
    ##P will hold the Prufer sequence
    P = []
    
    ##u is a counting variable
    u = 0
    while u < len(T):
        ##if u is a leaf (has exactly one adjacent vertex)
        if np.count_nonzero(T[u]) == 1:
            ##v will be the vertex adjacent to u
            for i in range(len(T)):
                if T[u][i] != 0:
                    v = i
                    break
            
            ##add v to the Prufer sequence
            P.append(v)
            
            ##remove the edge between u and v from T
            T[u][v], T[v][u] = 0, 0
            
            ##start checking for leaves again, at the beginning of the list
            u = 0
        
        else:
            ##if u is not a leaf, move on
            u = u + 1
    
    return P

### build_tree

#### Description

Given the Prufer sequence for a tree, construct and return the (unweighted) adjacency matrix of the tree.

#### Variables

*a* is the Prufer sequence for a tree.

In [3]:
def build_tree(a):
    ##n is the total number of leaves
    n = len(a) + 2
    
    ##make a list that records the degree of each vertex
    deg = [1 + a.count(i) for i in range(n)]
    
    ##T will store the adjacency matrix of the tree
    T = np.zeros((n, n))
    
    ##starting with the lowest numbered leaf, add an edge to the vertices given by the Prufer sequence
    for i in a:
        for j in range(n):
            if deg[j] == 1:
                T[i][j] = 1
                T[j][i] = 1
                deg[j] = deg[j] - 1
                deg[i] = deg[i] - 1
                break
    
    ##now there are just two vertices with positive degree remaining
    u = -1
    v = -1
    for i in range(n):
        if deg[i] == 1:
            if u == -1:
                u = i
            else:
                v = i
                break
    
    T[u][v], T[v][u] = 1, 1
    
    return T

### gen_tri_tree

#### Description

Given the number of leaves, *l*, return a random trivalent tree as a weighted adjacency matrix, which is equidistant if *eq* is true.

#### Variables

*l* a positive integer (will be the number of leaves of the tree returned).

*eq* a boolean indicating if the tree will be equidistant. True by default.

In [4]:
def gen_tri_tree(l, eq=True):
    ##n stores the total number of vertices in the tree
    ##a trivalent tree has l - 2 internal vertices, where l is the number of leaves
    n = l + (l - 2)
    
    ##the leaves are the first l numbers, internal vertices are the last l - 2
    leaves = [i for i in range(l)]
    internal = [i for i in range(l, n)]
    
    ##in a trivalent tree, each internal vertex appears in the Prufer sequence exactly twice
    ##the leaves don't appear at all
    perm = np.random.permutation(range(2*len(internal)))
    a = [internal[int[p/2]] for p in perm]
    
    ##now build the tree from the Prufer sequence we generated
    T = build_tree(a)
    
    ##if the tree needs to be equidistant, make it equidistant
    if eq:
        return make_eq(T)
    
    ##otherwise just return T
    return T

### make_eq(T)

#### Description

Given the adjacency matrix for a tree, return a new adjacency matrix which reweights the existing edges to make the tree equidistant.

#### Variables

*T* the (usually non-equidistant) adjacency matrix of a tree.

In [6]:
def make_eq(T):
    ##make a copy of the adjacency matrix
    ##we will use this to record the edges we have already checked
    T_copy = T.copy()
    ##store the number of vertices
    n = len(T)
    
    ##d will store the weighted adjacency matrix
    d = np.zeros((n, n))
    
    ##assumes the first (len(T) + 2)/2 vertices are the leaves
    leaves = [i for i in range(2, int((n + 2)/2))]
    
    ##find the internal node connected to 0
    u = -1
    for j in range(len(T)):
        if T[0][j] == 1:
            u = j
            ##remove the edge between 0 and u
            T_copy[0][u], T_copy[u][0] == 0, 0
            break
    
    d = find_lengths(T_copy, d, u, 1)
    
    ##add the edges back to 0
    ##this ensures the tree is trivalent, but 0 is the root of the tree
    d[0][u], d[u][0] = 1, 1
    
    return d

### find_lengths

#### Description

A recursive method that finds lengths to make a tree equidistant with a given height.

#### Variables

*T* is a matrix that records the unvisited edges of the given tree.

*d* is a matrix recording the lengths we assign to edges; this is what we return at the end.

*u* is a number representing the current node.

*l* is the current maximum height.

In [9]:
def find_lengths(T, d, u, l):
    ##T keeps track of edges we haven't visited
    ##if T is all zeros, then we've visited all edges, so we're done
    if np.count_nonzero(T) == 0:
        return d
    
    ##otherwise, get the two nodes connected to u
    v = -1
    w = -1
    for j in range(len(T)):
        ##do I need to worry about loops?
        if T[u][j] == 1:
            if v == -1:
                v = j
            else:
                w = j
                break
    
    ##get the degrees of the vertices connected to u
    deg_v = np.count_nonzero(T[v])
    deg_w = np.count_nonzero(T[w])
    
    ##remove the edges to u
    T[u][v], T[v][u], T[u][w], T[w][u] = 0, 0, 0, 0
    
    ##CASE 1: v, w are leaves
    if deg_v == 1 and deg_w == 1:
        d[u][v], d[v][u], d[u][w], d[w][u] = l, l, l, l
        return d
    
    ##CASE 2: v is a leaf and w is internal
    elif deg_v == 1 and deg_w == 3:
        d[u][v], d[v][u] = l
        
        lr = np.random.randrange(1, l-1)
        d[u][w], d[w][u] = lr, lr
        
        return findLengths(T, d, w, l-lr)
    
    ##CASE 3: v is internal and w is a leaf
    elif deg_v == 3 and deg_w == 1:
        d[u][w], d[w][u] = l
        
        lr = np.random.randrange(1, l-1)
        d[u][v], d[v][u] = lr, lr
        
        return findLengths(T, d, v, l-lr)
    
    ##CASE 4: both v and w are internal nodes
    else:
        l1 = np.random.randrange(1, l-1)
        d[u][v], d[v][u] = l1, l1
        
        l2 = np.random.randrange(1, l-1)
        d[u][w], d[w][u] = l2, l2
        
        d = findLengths(T, d, v, l-l1)
        return findLengths(T, d, w, l-l2)

### get_dist

#### Description

A recursive method that finds the distance between two leaves on a tree.

#### Variables

*D* is the weighted adjacency matrix of the tree.

*i*, *j* are the two vertices we want to find the distance between.

*d* keeps a running record of the distance.

In [11]:
def get_dist(D, i, j, d):
    ##if we reached j, we're done
    if i == j:
        return d
    
    ##n stores the number of vertices
    n = len(D)
    
    ##get a list of the vertices adjacent to i
    adjacent = [k for k in range(n) if D[i][k] != 0]
    
    ##if we reached a leaf that's not j, we can't go any further
    if len(adjacent) == 0:
        return -1
    
    temp_dist = D[i].copy()
    for k in adjacent:
        D[i][k], D[k][i] = 0, 0
        
    return max([get_dist(D, k, j, d + temp_dist[k]) for k in adjacent])

### get_metric

#### Description

Given the weighted adjacency matrix of a tree, return the metric on its leaves.

#### Variables

*D* is the weighted adjacency matrix of the tree.

In [12]:
def get_metric(D):
    ##n stores the total number of vertices
    n = len(D)
    
    ##a trivalent tree has (n + 2)/2 leaves
    num_leaves = int((n + 2)/2)
    
    ##the dictionary u stores the metric
    u = dict()
    
    for i in range(num_leaves):
        for j in range(i+1, num_leaves):
            dij = get_dist(D.copy(), i, j, 0)
            u[(i,j)] = dij
            u[(j,i)] = dij
    
    return u